<a href="https://colab.research.google.com/github/RamSankarTheDeveloper/Question-generation-algorithm-based-on-blooms-taxonomy/blob/preprocessing/preprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## requirements

### coref code

In [3]:
#note:restart the kernel if you had already imported another spacy version in collab
#date:21/02/2023
#-----------------------------------------
#https://youtu.be/Sp6jZFHI02Y
!git clone https://github.com/huggingface/neuralcoref.git 
import os
os.chdir("/content/neuralcoref")
!pip install -r requirements.txt
!pip install -e .  #'.' is reqd
#---------------------------------------
!pip install spacy==2.3.0 
import spacy #after restarting the kernel
print(spacy. __version__)
#-----------------------------------------
import spacy.cli 
spacy.cli.download("en")
#so that "en" works
#OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.
#https://stackoverflow.com/questions/62822737/oserror-e050-cant-find-model-de-it-doesnt-seem-to-be-a-shortcut-link-a
#-------------------------------------------
# Load your usual SpaCy model (one of SpaCy English models)
import spacy
nlp = spacy.load('en')

#Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp)

# You're done. You can now use NeuralCoref as you usually manipulate a SpaCy document annotations.

Cloning into 'neuralcoref'...
remote: Enumerating objects: 772, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 772 (delta 10), reused 16 (delta 7), pack-reused 748
Receiving objects: 100% (772/772), 67.85 MiB | 32.68 MiB/s, done.
Resolving deltas: 100% (407/407), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 774.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.2 MB/s eta 0:00:00
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.6
    Uninstalling srsly-2.4.6:
      Successfully uninstalled srsly-2.4.6
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting

100%|██████████| 40155833/40155833 [00:01<00:00, 28961339.60B/s]


### imports

In [4]:
import pandas as pd
import numpy as np
#import spacy
nlp1 = spacy.load("en_core_web_sm")
import nltk
nltk.download('all')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.stem import PorterStemmer
pst = PorterStemmer()
import re


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

## Preprocess

In [5]:
class preprocess:

  def convert_para_to_listofsents(self, text):
    doc=nlp1(text)
    token_sentences=[]
    string_sentences=[]
    for sent in doc.sents:
      token_sentences.append(sent)
      string_sentences.append(sent.text)
    return  string_sentences #,token_sentences,

  def convert_to_noun_phrases(self,sentence):
    doc =nlp1(sentence)
    a=[]
    for np in doc.noun_chunks:
      a.append(np.text)
    return a

  def convert_to_tokens(self,sentence):
    #nltk
    token_list = word_tokenize(sentence)
    #for i in range(len(token_list)):
     # token_list[i]=token_list[i].text
    return token_list

  def convert_to_stopwords(self,sentence):
    sp=spacy.load("en_core_web_sm")
    all_stopwords = sp.Defaults.stop_words
    text_tokens = word_tokenize(sentence)
    tokens_without_sw= [word for word in text_tokens if not word in all_stopwords]
    for i in tokens_without_sw:
      tokens_without_sw[i]=tokens_without_sw[i].text
    return tokens_without_sw

  def convert_to_basewords(self,sentence):
    a=[]
    doc=nlp1(sentence)
    for token in doc:
      a.append(token.lemma_)
    for i in range(len(a)):
      a[i]=pst.stem(a[i])
    return a
    
  def convert_listitems_to_lowercase(self,list1):
    for i in range(len(list1)):
      list1[i]=list1[i].cnvrt_to_lwr()
    return list1

  def replace_all_corefs(self,text,k=0): 
    #change it into different versions
    #https://stackoverflow.com/questions/6570635/installing-multiple-versions-of-a-package-with-pip
    doc = nlp(text)
    edited_text = doc._.coref_resolved
    nouns = doc._.coref_clusters
    items=nouns
    for i in range(len(nouns)):
      nouns[i]=str(nouns[i]).split(":")[0]
    if k == 0:
      return edited_text
    else :
      return edited_text , items

  def convert_to_ngrams(self,sentence, n):
    manygrams = list(ngrams(sentence.split(), n))
    for i in manygrams:
      manygrams[i]=manygrams[i].text
    return manygrams
  


## Level 1

### code to point, extract the entities in text and make 'fill in the blanks' type questions.

In [6]:
class l1:

  def __init__ (self, paragraph= "India is stronger than Pakistan. Pakistan is stronger than Afghanistan", inprint=""):
    
    self.paragraph = paragraph
    self.fill_in_the_blanks_list = []
    self.coref_text = preprocess().replace_all_corefs(paragraph)
    self.doc = nlp1(self.coref_text)
    
    self.dep()
    self.ent_list_of_current_sentence = []
    self.iterate_sentences()

    if inprint.lower() == "inprint":
      self.print_results()


  def dep(self):
    subj=root_verb=obj =[]
    for token in self.doc:
      if token.dep_ in ["nsubjpass",'csubjpass','nsubj','csubj']:
        subj.append(token)
      if token.dep_ in ["ROOT","root"]:
        root_verb.append(token)
      if token.dep_ in ["dobj",'iobj','oprd','obj',"pobj"]:
        obj.append(token)
      return subj,root_verb, obj
      
  def refresh_entity_list(self, sentence):
    self.ent_list_of_current_sentence = []
    doc = nlp(sentence)
    for ent in doc.ents:
      self.ent_list_of_current_sentence.append(ent)

  def iterate_sentences(self):
   sentences_list = preprocess().convert_para_to_listofsents(self.coref_text)
   for sentence in sentences_list:
     self.refresh_entity_list(sentence)
     words_in_sentence = preprocess().convert_to_tokens(sentence)
     self.iter_words(words_in_sentence)

  def iter_words(self, words_in_sentence):
    for word in words_in_sentence:
      for ent in self.ent_list_of_current_sentence:
        if str(ent).lower() == word.lower():
          self.question(words_in_sentence) 
          return
    
  def question(self, words_in_sentence):
    for ent in self.ent_list_of_current_sentence:
      question = []
      for i in range(len(words_in_sentence)):
        if words_in_sentence[i].lower() == ent.text.lower():
          question = words_in_sentence.copy()
          #wh_question_word = self.check_ent_label_type(ent.label_)
          blank = str(ent.label_)
          #print(question[i])
          #question[i] = wh_question_word
          question[i] = "________"
          each_tuple = (" ".join(question), ent)
          self.fill_in_the_blanks_list.append(each_tuple)

  def check_ent_label_type(self, ent):
    if ent in {"TIME","DATE"}:
      return "when"
    elif ent in {"PERSON","NORP"}:
      return "who"
    elif ent in {"FACILITY","FAC","GPE",'LOC',"GPE","GPE_LOC"}:
      return "where"
    elif ent in {"PRODUCT","EVENT","WORK_OF_ART","LAW","LANGUAGE","PROD","ORG","GPE_ORG"}:
      return "what"

  def print_results(self):
    if self.fill_in_the_blanks_list == []:
      print("level1: no entity found for fill in the blanks")
    else:
      print("============level1_fill in the blanks===========")
      for each_item in self.fill_in_the_blanks_list:
        print("=>", each_item[0], "  ( clue:", each_item[1].label_,")")


## Level 2

## Level 3

### code to find and extract content mentioning differences and creates questions out of  them syntactically

In [7]:
#this code inputs a paragraph and outputs a fill in the blank level questions. Use it by making an object with text as argument.

class cls_diff:
  
  def __init__(self, paragraph, inprint = ''):
    self.paragraph = paragraph
    self.difference_synonyms_string = "contrast difference distinction contrary unlike unlike versus vs"
    self.sentences_list=[]
    self.list_of_questions=[]
    self.between_splitted_sentence=""
    self.prepare_seperating_and_synonyms_words()
    self.prepare_paragraph_to_0basewords_of_sentences_listoflist0()
    self.extract_sentences_that_contains_synonyms()
    self.basewords_of_sentences_listoflist
    self.inprint = inprint

    if self. inprint.lower() == "inprint":
     self.print_questions()
    
  def prepare_seperating_and_synonyms_words(self):
    #prepare difference_synonyms_basewords_list and seperator_words for matching (check_0difference_synonyms_basewords_list0_present_in_0basewords_of_sentences_listoflist0():)
    obj_preprocess = preprocess()
    self.difference_synonyms_basewords_list = obj_preprocess.convert_to_basewords(self.difference_synonyms_string)
    self.seperator_words = obj_preprocess.convert_to_basewords("and versus vs ,")
    
  def prepare_paragraph_to_0basewords_of_sentences_listoflist0(self):
    #text preperation into basewords for matching (check_0difference_synonyms_basewords_list0_present_in_0basewords_of_sentences_listoflist0():)
    text = self.paragraph
    preprocess_obj = preprocess()
    coref_text = preprocess_obj.replace_all_corefs(text)
    self.sentences_list=self.sentences_list+preprocess_obj.convert_para_to_listofsents(text)
    basewords_of_sentences_listoflist=[]
    for each_sentence in self.sentences_list:
      basewords_of_sentences_listoflist.append(preprocess_obj.convert_to_basewords(each_sentence))
    self.basewords_of_sentences_listoflist = basewords_of_sentences_listoflist

  def extract_sentences_that_contains_synonyms(self):
    #returns the index if the synonym of "difference" is present in any of sentences. 
    # variables: #synonyms of the word "difference" = difference_synonyms_basewords_list , sentences = basewords_of_sentences_listoflist
    basewords_of_sentences_listoflist = self.basewords_of_sentences_listoflist
    for index in range(len(basewords_of_sentences_listoflist)):
      for each_synonym in self.difference_synonyms_basewords_list:
        if each_synonym in basewords_of_sentences_listoflist[index]:
          #return index
          self.split_sentences_with_between_etc(index)  #each iteration main function
          break;
          #print("index=",index)
          
  def split_sentences_with_between_etc(self, index):
    if "between" in self.sentences_list[index].lower():
      betw_splitted_sentence=str(self.sentences_list[index].split("between")[1])
      self.betw_splitted_sentence=betw_splitted_sentence
      betw_splitted_sentence_words_listofstrngs=betw_splitted_sentence.split()
      self.split_sententences_with_and_etc(betw_splitted_sentence)
        #regex

  def split_sententences_with_and_etc(self, betw_splitted_sentence):
    #test_list = ['and', 'versus']
    test_list = self.seperator_words
    #betw_splitted_sentence = 'the main difference between power of z angle, x angle and y  angle is very interesting to teach'
    # code to split the line with 'diff' with keywords and and comma
    if re.search(r'\b({})\b'.format("|".join(test_list)),betw_splitted_sentence):
      splitted_parts_lstofstrngs = re.sub("[\s]and[\s]", "~",betw_splitted_sentence ) #change splittd parts lst of strings to before and after since this is a string
      splitted_parts_lstofstrngs = re.sub(",[\s]", "~", splitted_parts_lstofstrngs)
      splitted_parts_lstofstrngs = re.sub("[\s]versus[\s]", "~", splitted_parts_lstofstrngs)
      splitted_parts_lstofstrngs = re.sub("[\s]vs[\s]", "~", splitted_parts_lstofstrngs)
      splitted_parts_lstofstrngs = re.split("~",splitted_parts_lstofstrngs)
      self.extract_dependency_from_splitted_parts(splitted_parts_lstofstrngs)

  def extract_dependency_from_splitted_parts(self, splitted_parts_lstofstrngs):
      #code to extract dependencies from parts splitted by seperating words.
      words_and_deps_ofsplit_asdict={}
      dep_pattern_ofallsplits_asstrng=""
      words_and_deps_ofsplits_aslistofdict=[]
      for each_splitted_part in splitted_parts_lstofstrngs:
        for token in word_tokenize(each_splitted_part):
          words_and_deps_ofsplit_asdict[nltk.pos_tag([token])[0][0]] = nltk.pos_tag([token])[0][1] #0=>word, 1=>dep
          dep_pattern_ofallsplits_asstrng=dep_pattern_ofallsplits_asstrng+str("<")+nltk.pos_tag([token])[0][1]+str(">")
        words_and_deps_ofsplits_aslistofdict.append(words_and_deps_ofsplit_asdict)
        dep_pattern_ofallsplits_asstrng=dep_pattern_ofallsplits_asstrng+"~"
        words_and_deps_ofsplit_asdict={}
      dep_pattern_ofallsplits_asstrng=dep_pattern_ofallsplits_asstrng[:-1]
      dep_pattern_ofallsplits_aslstofstrngs=dep_pattern_ofallsplits_asstrng.split("~")
      self.find_smallest_split_pattern(dep_pattern_ofallsplits_aslstofstrngs)

  def find_smallest_split_pattern(self, dep_pattern_ofallsplits_aslstofstrngs):
    #choose smallest part of seperated line parts
    smallest_diff_line=dep_pattern_ofallsplits_aslstofstrngs[0]
    for i in dep_pattern_ofallsplits_aslstofstrngs[1:]:
      if len(i)<len(smallest_diff_line):
        smallest_diff_line=i
    largest_common_pattern_of_diff=self.create_all_ordered_compinations_of_word(smallest_diff_line)
    self.match_deppattern_in_sentence(largest_common_pattern_of_diff)

  def create_all_ordered_compinations_of_word (self, pattern1): 
    #code to get each ordered compination of a sentence to check for a pattern from in descending order of length
    for incr in range(len(pattern1)): #pattern1 is named so,  because renaming "pattern" might change other keywords with "pattern" in it
      decr=len(pattern1)-incr-1
      decr_cp=decr
      #print(line[0:k])
      sub_incr=0
      while decr_cp !=len(pattern1):
        return pattern1
        #use regex, if present, output that pattern.
        #print(pattern1[sub_incr:decr_cp+1]) #activate this line to print all ordered combinations of word in the terminal
        decr_cp=decr_cp+1
        sub_incr=sub_incr+1

  def match_deppattern_in_sentence(self,pattern1):
    token = word_tokenize(self.betw_splitted_sentence)
    tags = nltk.pos_tag(token)
    reg = "NN:{" + str(pattern1) + "}"
    a = nltk.RegexpParser(reg)
    result = a.parse(tags)
    list_of_contexts_ineachsentence=[]
    for subtree in result.subtrees():
      a=" ".join(w for w, t in subtree.leaves())
      list_of_contexts_ineachsentence.append( a ) 
    self.create_question_and_save(list_of_contexts_ineachsentence[1:])

  def create_question_and_save(self, list_of_contexts_ineachsentence_cpy):
    for i in range(len(list_of_contexts_ineachsentence_cpy)-1):
      list_of_contexts_ineachsentence_cpy[i] = list_of_contexts_ineachsentence_cpy[i] + str(",")

    list_of_contexts_ineachsentence_cpy[-1] = str("and ") + list_of_contexts_ineachsentence_cpy[-1]
    
    self.list_of_questions.append(str("What are the differences between " + " ".join(list_of_contexts_ineachsentence_cpy))  +  str(" ?"))

  def print_questions(self):
    if self.list_of_questions ==[]:
      print("level3: no context regarding diffs was found")
    else:
      print("\n","<========level 3, context = difference=========>")
      for each_question in self.list_of_questions:
        print("=> ", each_question)
      print("=============================================")


    


In [8]:
#notes:=
  #diff_list=["contrast","difference","distinction","contrary","different","differing","unlike","differ","unlike","versus","vs","v/s"]
  #diff_maybe_list=["the opposite","the contrary","the reverse","compare","on the other hand","while","whereas","but"]
  #seperator_words=["and",",","versus","vs"]

### similarities

In [9]:
#similarity_list=["resemblance similarity parallels analogy similar, analogous, common, comparable, identical, same, resemble, correspond, similarly, likewise, same, like, as"]

## main program

In [58]:

text =  'The main difference between a good book and a bad book is its difference. The main difference between a well written story and a poorly written story is its story.'


l1(text,"inprint")
cls_diff("India is my country", "inprint")
text

level1: no entity found for fill in the blanks
level3: no context regarding diffs was found


'The main difference between a good book and a bad book is its difference. The main difference between a well written story and a poorly written story is its story.'

## experimenting codes section

### keybert

In [ ]:
pip install keybert


In [ ]:
################################################


# script.py
from keybert import KeyBERT

doc = """
    Supervised learning is the machine learning task of learning a function that
    maps an input to an output based on example input-output pairs. It infers a
    function from labeled training data consisting of a set of training examples.
    In supervised learning, each example is a pair consisting of an input object
    (typically a vector) and a desired output value (also called the supervisory signal). 
    A supervised learning algorithm analyzes the training data and produces an inferred function, 
    which can be used for mapping new examples. An optimal scenario will allow for the 
    algorithm to correctly determine the class labels for unseen instances. This requires 
    the learning algorithm to generalize from the training data to unseen situations in a 
    'reasonable' way (see inductive bias).
"""

kw_model = KeyBERT()
keywords = kw_model.extract_keywords(doc)

print(kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None))
#[
#    ('learning', 0.4604),
#    ('algorithm', 0.4556),
#    ('training', 0.4487),
#    ('class', 0.4086),
#    ('mapping', 0.3700)
#]

[('supervised', 0.6676), ('labeled', 0.4896), ('learning', 0.4813), ('training', 0.4134), ('labels', 0.3947)]


### paraphrase

In [1]:
! pip install git+https://github.com/PrithivirajDamodaran/Parrot.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PrithivirajDamodaran/Parrot.git to /tmp/pip-req-build-rods5xau
  Running command git clone --filter=blob:none --quiet https://github.com/PrithivirajDamodaran/Parrot.git /tmp/pip-req-build-rods5xau
  Resolved https://github.com/PrithivirajDamodaran/Parrot.git to commit 720a87a1ee557d8ed8d9a021adbdd1dd5616c5f9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.0/174.0 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.6 MB/s eta 0:00:00


In [2]:
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

In [3]:
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [4]:
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

In [21]:
phrases = key_sentences_series

In [22]:
sentence_paraphrases_list=[]

In [7]:
for phrase in phrases:
  sentence_paraphrases_list = []
  sentence_paraphrase_list.append(phrase)
  para_phrases = parrot.augment(input_phrase=phrase)
  for para_phrase in para_phrases:
   sentence_paraphrases_list.append(para_phrase)
  sentence_paraphrase_list.append(sentence_paraphrases_list)

KeyboardInterrupt: ignored

In [30]:
for phrase in phrases:
  paraphrases = parrot.augment(input_phrase=phrase)
  for paraphrase in paraphrases:
   row = str(phrase) + str("=>paraphrase=>")+ str(paraphrase[0])
   sentence_paraphrases_list.append(row)

In [53]:
import re
sentence_paraphrases_list
sample_sentence = sentence_paraphrases_list.copy()
pattern = "[0-9][0-9]|\'\,|\(|\)"
for i in range(len(sample_sentence)):
  sample_sentence[i] = re.sub(pattern, "", sample_sentence[i])
  sample_sentence[i] = re.sub("\>\'", ">", sample_sentence[i])

sample_sentence

['There is still a contrast between East and West Germany.=>paraphrase=>east and west germany still face a contrast ',
 'There is still a contrast between East and West Germany.=>paraphrase=>there is a contrast between east germany and west germany ',
 'There is still a contrast between East and West Germany.=>paraphrase=>there is a contrast between west and east germany ',
 'There is still a contrast between East and West Germany.=>paraphrase=>there is still a difference between east and west germany ',
 'There is still a contrast between East and West Germany.=>paraphrase=>there is still a contrast between east and west germany ',
 'There is still a contrast between South, East and West Germany.=>paraphrase=>there is still a contrast between the west and south east german countries ',
 'There is still a contrast between South, East and West Germany.=>paraphrase=>there remains a contrast between the region of south east and west germany ',
 'There is still a contrast between South, Ea

In [54]:
x = []
for row in sample_sentence:
  x=x+row.split("=>paraphrase=>")
print(x)

['There is still a contrast between East and West Germany.', 'east and west germany still face a contrast ', 'There is still a contrast between East and West Germany.', 'there is a contrast between east germany and west germany ', 'There is still a contrast between East and West Germany.', 'there is a contrast between west and east germany ', 'There is still a contrast between East and West Germany.', 'there is still a difference between east and west germany ', 'There is still a contrast between East and West Germany.', 'there is still a contrast between east and west germany ', 'There is still a contrast between South, East and West Germany.', 'there is still a contrast between the west and south east german countries ', 'There is still a contrast between South, East and West Germany.', 'there remains a contrast between the region of south east and west germany ', 'There is still a contrast between South, East and West Germany.', 'there remains a contrast between south east germany a

In [57]:
print(type(x[0::2]))

<class 'list'>


In [60]:
import pandas as pd
df2 = pd.DataFrame(x[1::2],x[0::2])
df2

,0
There is still a contrast between East and West Germany.,east and west germany still face a contrast
There is still a contrast between East and West Germany.,there is a contrast between east germany and w...
There is still a contrast between East and West Germany.,there is a contrast between west and east germ...
There is still a contrast between East and West Germany.,there is still a difference between east and w...
There is still a contrast between East and West Germany.,there is still a contrast between east and wes...
...,...
"Contrary to popular opinion, immigrants are not driving down wages across the board.",contrary to popular opinion immigrants are not...
The present research results suggest the opposite of what scholars have maintained for more than a century.,the present research results suggest the oppos...
The present research results suggest the opposite of what scholars have maintained for more than a century.,the present research results suggest the oppos...
The present research results suggest the opposite of what scholars have maintained for more than a century.,the present research results suggest that the ...


In [ ]:
sample_sentence 

In [37]:
sample_sentence

["There is still a contrast between East and West Germany.=>paraphrase=>('east and west germany still face a contrast', 50)",
 "There is still a contrast between East and West Germany.=>paraphrase=>('there is a contrast between east germany and west germany', 27)",
 "There is still a contrast between East and West Germany.=>paraphrase=>('there is a contrast between west and east germany', 23)",
 "There is still a contrast between East and West Germany.=>paraphrase=>('there is still a difference between east and west germany', 22)",
 "There is still a contrast between East and West Germany.=>paraphrase=>('there is still a contrast between east and west germany', 13)",
 "There is still a contrast between South, East and West Germany.=>paraphrase=>('there is still a contrast between the west and south east german countries', 35)",
 "There is still a contrast between South, East and West Germany.=>paraphrase=>('there remains a contrast between the region of south east and west germany', 34

### creating data to train model

In [8]:
text_differences = """ 
There is still a contrast between East and West Germany.
East and West germany

There is still a contrast between South, East and West Germany.
South, East and West Germany

There is still a contrast between North, South, East and West Germany.
North, South, East and West Germany

Hamburg provides a contrast to its rather rural surrounding areas.
Hamburg, rural surrounding areas

There are differences in employment rates in different parts of the United Kingdom.
employment rates in different parts of the United Kingdom

Sociologists have long observed a difference in the living conditions of whites and Blacks in the US.
living conditions of whites and Blacks in the US

Sociologists have long observed a difference in the living conditions of white communities and Black communities in the US.
living conditions of white communities and Black communities in the US

It is reasonable to draw a distinction between biological sex and social gender identities.
biological sex and social gender identities

There appears to be a clear distinction between oral and written literature.
oral and written literature

I will begin this essay by examining two contrasting views on the Vietnam War.
views on the Vietnam War

A rather different approach was taken by Angela Davis, a counterculture activist who is a professor at the USCS in its History of Consciousness Department.
approach was taken by Angela Davis

Lakoff’s approach to similarities among languages is distinctly different from that of Chomsky.
Lakoff’s approach, Chomsky

The two examples reflect widely differing perspectives.
two examples

The current situation at the Mexico-United States border is very much unlike that of the borders within the European Union.
Mexico-United States border, borders within the European Union

The results run entirely contrary to our expectations.
results, expectations

His analysis contrasts with how most other scholars view this.
His analysis, how most other scholars view this

While the two claimed to have incompatible views, in fact, their opinions only differed slightly.
incompatible views

Her opinion becomes more comprehensible when compared to Miller’s.
Her opinion, Miller’s

It is important to distinguish between nationalism and patriotism. 
nationalism and patriotism

In this paper, I will distinguish between three analytical categories.
three analytical categories

Her analytical precision distinguishes her from other scholars. 
her from other scholars

In the first chapter, I will differentiate between social and economic factors.
social and economic factors

It is difficult to differentiate this product from its predecessor.
this product from its predecessor

An estimated 31 to 42% of the Indian population eat a vegetarian diet. In contrast, only approximately 5% of US Americans prefer not to eat meat.
Indian population eat a vegetarian diet, US Americans prefer not to eat meat

75 % of the working population in Germany are employed in tertiary sector. By comparison, only 33% of the working population in Vietnam is active in the services sector.
working population in Germany, working population in Vietnam

One of the main focuses of Adeyemi is the social implications of genetic testing. Nguyen, on the other hand, concentrates on the ethical implications.
Adeyemi, Nguyen

Unlike public sector agencies, NGOs have a tradition of setting up cooperation networks.
public sector agencies, NGOs

Socialists view freedom as a concrete situation as opposed to a purely abstract ideal. 
freedom as a concrete situation, to a purely abstract ideal

In contrast to various other experts on the subject, Bleyhl maintains that French teachers should not focus on grammar.
various other experts on the subject, Bleyhl

Small and mid-sized enterprises bear an administrative burden that is disproportionate in comparison with big enterprises.
Small and mid-sized enterprises, big enterprises

Regarding the issue of Christianity versus Islam there is a major problem. 
Christianity, Islam

The majority of students of French are women, whereas the majority of academics teaching it are men.
students of French, academics teaching it

While French is considered to be a prestigious language by the majority of the European population, the reverse is true for Arabic.
French, Arabic

It was widely believed that gods created the man. Scientists have proven the contrary to be true.
gods created the man, contrary

It was widely believed that gods created the man. Scientists have proven the contrary to be true.
gods created the man, contrary

Contrary to popular opinion, immigrants are not driving down wages across the board.
popular opinion, contrary

The present research results suggest the opposite of what scholars have maintained for more than a century.
present research results, what scholars have maintained for more than a century

"""

In [9]:
text_differences = text_differences.split("\n")
text_differences

[' ',
 'There is still a contrast between East and West Germany.',
 'East and West germany',
 '',
 'There is still a contrast between South, East and West Germany.',
 'South, East and West Germany',
 '',
 'There is still a contrast between North, South, East and West Germany.',
 'North, South, East and West Germany',
 '',
 'Hamburg provides a contrast to its rather rural surrounding areas.',
 'Hamburg, rural surrounding areas',
 '',
 'There are differences in employment rates in different parts of the United Kingdom.',
 'employment rates in different parts of the United Kingdom',
 '',
 'Sociologists have long observed a difference in the living conditions of whites and Blacks in the US.',
 'living conditions of whites and Blacks in the US',
 '',
 'Sociologists have long observed a difference in the living conditions of white communities and Black communities in the US.',
 'living conditions of white communities and Black communities in the US',
 '',
 'It is reasonable to draw a distinc

In [15]:
for i in text_differences:
  if len(i) ==0:
    text_differences.remove(i)
  if i == ' ':
    text_differences.remove(i)
x_basic_sentences_listofstrings = text_differences.copy()

In [16]:
x_basic_sentences_listofstrings

['There is still a contrast between East and West Germany.',
 'East and West germany',
 'There is still a contrast between South, East and West Germany.',
 'South, East and West Germany',
 'There is still a contrast between North, South, East and West Germany.',
 'North, South, East and West Germany',
 'Hamburg provides a contrast to its rather rural surrounding areas.',
 'Hamburg, rural surrounding areas',
 'There are differences in employment rates in different parts of the United Kingdom.',
 'employment rates in different parts of the United Kingdom',
 'Sociologists have long observed a difference in the living conditions of whites and Blacks in the US.',
 'living conditions of whites and Blacks in the US',
 'Sociologists have long observed a difference in the living conditions of white communities and Black communities in the US.',
 'living conditions of white communities and Black communities in the US',
 'It is reasonable to draw a distinction between biological sex and social ge

In [17]:
key_sentences_series = x_basic_sentences_listofstrings[0::2]
output_series = x_basic_sentences_listofstrings[1::2]
output_series

['East and West germany',
 'South, East and West Germany',
 'North, South, East and West Germany',
 'Hamburg, rural surrounding areas',
 'employment rates in different parts of the United Kingdom',
 'living conditions of whites and Blacks in the US',
 'living conditions of white communities and Black communities in the US',
 'biological sex and social gender identities',
 'oral and written literature',
 'views on the Vietnam War',
 'approach was taken by Angela Davis',
 'Lakoff’s approach, Chomsky',
 'two examples',
 'Mexico-United States border, borders within the European Union',
 'results, expectations',
 'His analysis, how most other scholars view this',
 'incompatible views',
 'Her opinion, Miller’s',
 'nationalism and patriotism',
 'three analytical categories',
 'her from other scholars',
 'social and economic factors',
 'this product from its predecessor',
 'Indian population eat a vegetarian diet, US Americans prefer not to eat meat',
 'working population in Germany, working p

In [18]:
key_sentences_series

['There is still a contrast between East and West Germany.',
 'There is still a contrast between South, East and West Germany.',
 'There is still a contrast between North, South, East and West Germany.',
 'Hamburg provides a contrast to its rather rural surrounding areas.',
 'There are differences in employment rates in different parts of the United Kingdom.',
 'Sociologists have long observed a difference in the living conditions of whites and Blacks in the US.',
 'Sociologists have long observed a difference in the living conditions of white communities and Black communities in the US.',
 'It is reasonable to draw a distinction between biological sex and social gender identities.',
 'There appears to be a clear distinction between oral and written literature.',
 'I will begin this essay by examining two contrasting views on the Vietnam War.',
 'A rather different approach was taken by Angela Davis, a counterculture activist who is a professor at the USCS in its History of Consciousne

In [19]:
import pandas as pd
x_basic_sentences_dataframe = pd.DataFrame(output_series, key_sentences_series)
x_basic_sentences_dataframe

,0
There is still a contrast between East and West Germany.,East and West germany
"There is still a contrast between South, East and West Germany.","South, East and West Germany"
"There is still a contrast between North, South, East and West Germany.","North, South, East and West Germany"
Hamburg provides a contrast to its rather rural surrounding areas.,"Hamburg, rural surrounding areas"
There are differences in employment rates in different parts of the United Kingdom.,employment rates in different parts of the Uni...
Sociologists have long observed a difference in the living conditions of whites and Blacks in the US.,living conditions of whites and Blacks in the US
Sociologists have long observed a difference in the living conditions of white communities and Black communities in the US.,living conditions of white communities and Bla...
It is reasonable to draw a distinction between biological sex and social gender identities.,biological sex and social gender identities
There appears to be a clear distinction between oral and written literature.,oral and written literature
I will begin this essay by examining two contrasting views on the Vietnam War.,views on the Vietnam War


In [20]:

key_sentences_series

['There is still a contrast between East and West Germany.',
 'There is still a contrast between South, East and West Germany.',
 'There is still a contrast between North, South, East and West Germany.',
 'Hamburg provides a contrast to its rather rural surrounding areas.',
 'There are differences in employment rates in different parts of the United Kingdom.',
 'Sociologists have long observed a difference in the living conditions of whites and Blacks in the US.',
 'Sociologists have long observed a difference in the living conditions of white communities and Black communities in the US.',
 'It is reasonable to draw a distinction between biological sex and social gender identities.',
 'There appears to be a clear distinction between oral and written literature.',
 'I will begin this essay by examining two contrasting views on the Vietnam War.',
 'A rather different approach was taken by Angela Davis, a counterculture activist who is a professor at the USCS in its History of Consciousne